# Bài 2 – Vấn đề xây dựng Content-Based Data (3 điểm)

## 1.Đọc dữ liệu & chọn 100 phim có nhiều thể loại nhất

In [11]:
import pandas as pd
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer

# ===== Đọc dữ liệu MovieLens =====
movies = pd.read_csv('../data/movie_lens/movies.csv')
# Cấu trúc: movieId, title, genres

# Chuyển genres thành danh sách
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))

# Đếm số thể loại mỗi phim
movies['num_genres'] = movies['genres'].apply(len)

# Lấy top 100 phim có nhiều thể loại nhất
top100 = movies.sort_values('num_genres', ascending=False).head(100).reset_index(drop=True)
print("🎬 Top 5 phim có nhiều thể loại nhất:")
print(top100[['title', 'genres', 'num_genres']].head())


🎬 Top 5 phim có nhiều thể loại nhất:
                                               title  \
0                                      Rubber (2010)   
1  Patlabor: The Movie (Kidô keisatsu patorebâ: T...   
2          Aelita: The Queen of Mars (Aelita) (1924)   
3                                       Mulan (1998)   
4                                      Robots (2005)   

                                              genres  num_genres  
0  [Action, Adventure, Comedy, Crime, Drama, Film...          10  
1  [Action, Animation, Crime, Drama, Film-Noir, M...           8  
2  [Action, Adventure, Drama, Fantasy, Romance, S...           7  
3  [Adventure, Animation, Children, Comedy, Drama...           7  
4  [Adventure, Animation, Children, Comedy, Fanta...           7  


## 2.Tính độ tương đồng Jaccard giữa các phim

- Jaccard = |Giao nhau| / |Hợp nhau|

In [12]:
# Mã hoá one-hot các thể loại bằng MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(top100['genres'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=top100['title'])

# Chọn 1 phim X để tính tương đồng
target_film = top100['title'][0]
target_vector = genre_df.loc[target_film]

# Tính Jaccard với từng phim
similarities = []
for title, row in genre_df.iterrows():
    if title != target_film:
        sim = jaccard_score(target_vector, row)
        similarities.append((title, sim))

# Lấy top 10 phim tương tự
top10_similar = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

print(f"\n🎯 Top 10 phim tương đồng nhất với '{target_film}' theo Jaccard:")
for title, score in top10_similar:
    print(f"{title:60}  →  {score:.3f}")



🎯 Top 10 phim tương đồng nhất với 'Rubber (2010)' theo Jaccard:
Seven-Per-Cent Solution, The (1976)                           →  0.600
Ichi the Killer (Koroshiya 1) (2001)                          →  0.600
Sherlock Holmes: A Game of Shadows (2011)                     →  0.600
Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)  →  0.500
Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003)                   →  0.500
Copycat (1995)                                                →  0.500
Suicide Kings (1997)                                          →  0.500
Sin City (2005)                                               →  0.500
11:14 (2003)                                                  →  0.500
Cirque du Freak: The Vampire's Assistant (2009)               →  0.455


## 3.Tạo user profile cho user Y xem 5 phim


- Ý tưởng: hồ sơ người dùng = trung bình các vector thể loại của phim đã xem.

In [13]:
import numpy as np

# Giả sử user Y đã xem 5 phim đầu tiên trong top100
user_watched = top100['title'][:5].tolist()
print("\n👤 User Y đã xem:", user_watched)

# Trung bình các vector genres
user_profile = genre_df.loc[user_watched].mean()

# Tính Jaccard giữa user profile và các phim khác
scores = []
for title, row in genre_df.iterrows():
    if title not in user_watched:
        sim = jaccard_score(user_profile > 0.5, row)  # nhị phân hoá
        scores.append((title, sim))

# Gợi ý top 10 phim
user_top10 = sorted(scores, key=lambda x: x[1], reverse=True)[:10]
print("\n🎬 Gợi ý top 10 phim cho user Y:")
for title, score in user_top10:
    print(f"{title:60}  →  {score:.3f}")



👤 User Y đã xem: ['Rubber (2010)', 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)', 'Aelita: The Queen of Mars (Aelita) (1924)', 'Mulan (1998)', 'Robots (2005)']

🎬 Gợi ý top 10 phim cho user Y:
Teenage Mutant Ninja Turtles: Turtles Forever (2009)          →  0.714
Chicken Little (2005)                                         →  0.625
Maximum Ride (2016)                                           →  0.625
Kingsglaive: Final Fantasy XV (2016)                          →  0.625
Ratchet & Clank (2016)                                        →  0.625
Dragonheart 2: A New Beginning (2000)                         →  0.625
Meet the Robinsons (2007)                                     →  0.625
Stunt Man, The (1980)                                         →  0.625
Futurama: Bender's Game (2008)                                →  0.625
Mars Needs Moms (2011)                                        →  0.556


## 4.Biến thể Cosine Similarity, Dice, Overlap (thay Jaccard)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sim_matrix = cosine_similarity(genre_df)
sim_df = pd.DataFrame(sim_matrix, index=genre_df.index, columns=genre_df.index)

target = top100.loc[0, 'title']
print("\n🎬 Top 10 phim tương tự (Cosine):")
print(sim_df[target].sort_values(ascending=False).iloc[1:11])



🎬 Top 10 phim tương tự (Cosine):
title
Seven-Per-Cent Solution, The (1976)                               0.774597
Sherlock Holmes: A Game of Shadows (2011)                         0.774597
Ichi the Killer (Koroshiya 1) (2001)                              0.774597
11:14 (2003)                                                      0.707107
Copycat (1995)                                                    0.707107
Sin City (2005)                                                   0.707107
Suicide Kings (1997)                                              0.707107
Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003)                       0.707107
Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)    0.670820
Northwest Passage (1940)                                          0.645497
Name: Rubber (2010), dtype: float64


In [15]:
import numpy as np
import pandas as pd

# Giả sử genre_df là DataFrame nhị phân: mỗi hàng = phim, mỗi cột = thể loại
# Ví dụ:
# genre_df.head()
#       Action  Comedy  Drama  Romance  ...
# film1     1       0      1       0
# film2     0       1      0       1

# ===== DICE SIMILARITY =====
def dice_similarity_matrix(df):
    X = df.values
    intersection = np.dot(X, X.T)
    sums = X.sum(axis=1)
    dice = (2 * intersection) / (sums[:, None] + sums[None, :])
    np.fill_diagonal(dice, 1.0)
    return pd.DataFrame(dice, index=df.index, columns=df.index)

# ===== OVERLAP SIMILARITY =====
def overlap_similarity_matrix(df):
    X = df.values
    intersection = np.dot(X, X.T)
    sums = X.sum(axis=1)
    overlap = intersection / np.minimum(sums[:, None], sums[None, :])
    np.fill_diagonal(overlap, 1.0)
    return pd.DataFrame(overlap, index=df.index, columns=df.index)

# Tính hai ma trận tương đồng
dice_df = dice_similarity_matrix(genre_df)
overlap_df = overlap_similarity_matrix(genre_df)

target = top100.loc[0, 'title']

print("\n🎬 Top 10 phim tương tự (Dice):")
print(dice_df[target].sort_values(ascending=False).iloc[1:11])

print("\n🎬 Top 10 phim tương tự (Overlap):")
print(overlap_df[target].sort_values(ascending=False).iloc[1:11])



🎬 Top 10 phim tương tự (Dice):
title
Seven-Per-Cent Solution, The (1976)                               0.750000
Sherlock Holmes: A Game of Shadows (2011)                         0.750000
Ichi the Killer (Koroshiya 1) (2001)                              0.750000
11:14 (2003)                                                      0.666667
Copycat (1995)                                                    0.666667
Sin City (2005)                                                   0.666667
Suicide Kings (1997)                                              0.666667
Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)    0.666667
Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003)                       0.666667
Northwest Passage (1940)                                          0.625000
Name: Rubber (2010), dtype: float64

🎬 Top 10 phim tương tự (Overlap):
title
Seven-Per-Cent Solution, The (1976)            1.000000
Sherlock Holmes: A Game of Shadows (2011)      1.000000
Ichi the Killer (Korosh

## 5.Profile theo trọng số rating (Weighted User Profile)

In [16]:
# Giả định rating ngẫu nhiên 1–5 cho 5 phim đã xem
import numpy as np
ratings = np.random.randint(3, 6, size=5)
user_profile_w = (genre_df.loc[user_watched].T * ratings).T.sum() / ratings.sum()

recs = []
for title, row in genre_df.iterrows():
    if title not in user_watched:
        sim = cosine_similarity([user_profile_w], [row])[0][0]
        recs.append((title, sim))

print("\n⭐ Top 10 phim cho user có rating trọng số:")
for t, s in sorted(recs, key=lambda x: x[1], reverse=True)[:10]:
    print(f"{t:75} → {s:.3f}")



⭐ Top 10 phim cho user có rating trọng số:
Stunt Man, The (1980)                                                       → 0.776
Interstate 60 (2002)                                                        → 0.774
Dragonheart 2: A New Beginning (2000)                                       → 0.767
Kingsglaive: Final Fantasy XV (2016)                                        → 0.750
Osmosis Jones (2001)                                                        → 0.742
Getaway, The (1994)                                                         → 0.733
Aqua Teen Hunger Force Colon Movie Film for Theaters (2007)                 → 0.726
Seven-Per-Cent Solution, The (1976)                                         → 0.724
Ernest & Célestine (Ernest et Célestine) (2012)                             → 0.724
Inside Out (2015)                                                           → 0.715


## 6.Biến thể TF-IDF theo mô tả phim (text)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

movies = pd.read_csv("../data/movie_dataset.csv")  # cột: title, overview
movies = movies.dropna(subset=['overview'])
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(movies['overview'])

sim = cosine_similarity(tfidf)
sim_df = pd.DataFrame(sim, index=movies['title'], columns=movies['title'])

film = movies['title'].iloc[0]
print(f"\n🧠 Top 10 phim tương tự về mô tả với {film}:")
print(sim_df[film].sort_values(ascending=False).iloc[1:11])



🧠 Top 10 phim tương tự về mô tả với Avatar:
title
Apollo 18                       0.186806
The American                    0.170153
The Matrix                      0.132308
The Inhabited Island            0.123932
Tears of the Sun                0.122487
Hanna                           0.104698
The Adventures of Pluto Nash    0.100915
Semi-Pro                        0.097997
Supernova                       0.094480
Blood and Chocolate             0.093139
Name: Avatar, dtype: float64


## 7.Hybrid (Content + Popularity)

In [18]:
# Giả định rating trung bình cho mỗi phim
import numpy as np
top100['avg_rating'] = np.random.uniform(3,5,len(top100))
alpha = 0.6

# Tính lại ma trận tương đồng cho top100
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix_top100 = cosine_similarity(genre_df)
sim_df_top100 = pd.DataFrame(sim_matrix_top100, index=genre_df.index, columns=genre_df.index)

hybrid_score = alpha * sim_df_top100[target_film] + (1-alpha) * (top100.set_index('title')['avg_rating'] / 5)
print("\n⚗️ Top 10 phim Hybrid (Content + Popularity):")
print(hybrid_score.sort_values(ascending=False).iloc[1:11])



⚗️ Top 10 phim Hybrid (Content + Popularity):
title
Seven-Per-Cent Solution, The (1976)                                                0.799762
Lupin III: The Castle Of Cagliostro (Rupan sansei: Kariosutoro no shiro) (1979)    0.783940
Gozu (Gokudô kyôfu dai-gekijô: Gozu) (2003)                                        0.775185
Mummy, The (1999)                                                                  0.772739
Suicide Kings (1997)                                                               0.769612
Cave, The (2005)                                                                   0.769135
Chase, The (1994)                                                                  0.756466
Ichi the Killer (Koroshiya 1) (2001)                                               0.750699
Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)                     0.748833
Sin City (2005)                                                                    0.737628
dtype: float64


## 8.Cluster-based Recommendation (K-means trên genres)

In [19]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0)
labels = kmeans.fit_predict(genre_df)
top100['cluster'] = labels

target_cluster = top100.loc[0, 'cluster']
cluster_movies = top100[top100['cluster'] == target_cluster]['title'].tolist()[:10]
print(f"\n🧩 10 phim cùng nhóm thể loại với {target_film}:")
print(cluster_movies)



🧩 10 phim cùng nhóm thể loại với Rubber (2010):
['Rubber (2010)', 'Aelita: The Queen of Mars (Aelita) (1924)', 'Interstate 60 (2002)', 'Osmosis Jones (2001)', "Cirque du Freak: The Vampire's Assistant (2009)", 'Chase, The (1994)', 'Young Sherlock Holmes (1985)', 'Northwest Passage (1940)', 'Maximum Ride (2016)', 'Green Hornet, The (2011)']


c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\thong\AppData\Local\Programs\Python\Python310\lib\subp

## 9.Graph-based Recommendation (NetworkX)

In [20]:
import networkx as nx
G = nx.Graph()

# Tạo cạnh giữa phim và thể loại
for title, genres in zip(top100['title'], top100['genres']):
    for g in genres:
        G.add_edge(title, g)

target = top100['title'][2]
neighbors = list(nx.neighbors(G, target))
# Phim khác có chung thể loại
similar_movies = [n for n in neighbors if n in top100['title'].values]
print(f"\n🌐 Phim liên kết qua thể loại với {target}: ", similar_movies[:10])



🌐 Phim liên kết qua thể loại với Aelita: The Queen of Mars (Aelita) (1924):  []


## 10.Cold-start Item – thêm phim mới và xử lý bằng content

In [21]:
new_movie = {'title': 'New Sci-Fi Action',
             'genres': ['Action','Sci-Fi','Adventure']}
new_vec = pd.Series({g:1 if g in new_movie['genres'] else 0 for g in mlb.classes_})
genre_df = pd.concat([genre_df, new_vec.to_frame().T])
genre_df.index = list(genre_df.index[:-1]) + [new_movie['title']]

sim = cosine_similarity(genre_df)
sim_df = pd.DataFrame(sim, index=genre_df.index, columns=genre_df.index)
print("\n🆕 Phim mới tương tự nhất:")
print(sim_df.loc['New Sci-Fi Action'].sort_values(ascending=False).iloc[1:6])



🆕 Phim mới tương tự nhất:
Ant-Man and the Wasp (2018)          0.774597
Super Mario Bros. (1993)             0.707107
Batman/Superman Movie, The (1998)    0.707107
Chicken Little (2005)                0.707107
Ratchet & Clank (2016)               0.707107
Name: New Sci-Fi Action, dtype: float64


## 11.Evaluation Precision@K (ví dụ mô phỏng)

In [ ]:
# Giả định user đã thích 3 phim trong 10 phim được gợi ý
recommended = [t for t,_ in user_top10]
relevant = recommended[:3]     # giả định 3 phim đầu đúng sở thích

precision_at_k = len(relevant)/len(recommended)
print(f"\n📊 Precision@10 = {precision_at_k:.2f}")



📊 Precision@10 = 0.30


: 